In [207]:
# Import the libraries needed
from bs4 import BeautifulSoup, SoupStrainer
from collections import Counter
# import httplib2
import itertools
import matplotlib as plt
import nltk
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
import pandas as pd 
import random
import spacy
from spacy import displacy
from spacy.lang.en import English
import urllib.request
from urllib.request import urlopen, Request
import random
import re
import requests

In [208]:
nlp = spacy.load("en_core_web_sm")

## Importing Dataset

In [209]:
reviews = pd.read_json('../data/reviewSelected100.json', encoding='ISO-8859-1', lines=True)

## 3.2 Dataset Analysis

### Tokenisation and Stemming

In [210]:
# get reviews for a random business 
random_business = reviews.sample()
random_business_id = random_business.iloc[0]['business_id']
small_business_dataset = reviews.loc[reviews['business_id'] == random_business_id]
small_business_dataset.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
3112,1WhT1Fp4YcQqc9pOOnd2Iw,akL4MRM2jytzX07Ut2HDyA,NdpvGGF4cLrdnA6ydSZz3g,1,1,0,0,Service was horrible. Simply way overcharging ...,2017-12-29 15:58:00
3159,l8s3iaWZUEsp1Y7E8aVlDA,i0cM1FAC3dy3t5tReCm98Q,NdpvGGF4cLrdnA6ydSZz3g,3,2,0,0,"-Each 'order' of meat is literally 5 slices, s...",2016-02-23 23:30:05
3181,AOV9NUzFiXxQOKcHKflN_w,bjy0OVaWhdDh3vynEyNNtg,NdpvGGF4cLrdnA6ydSZz3g,5,0,0,0,This restaurant is very good. I love the servi...,2017-01-28 21:57:40
3192,BxD9tgEvdE7pk3FVh_pFHg,4fdrCA2ahO2LA4G39hVdlw,NdpvGGF4cLrdnA6ydSZz3g,4,0,0,0,Very good service and variety of foods you can...,2017-05-14 16:24:44
3193,ljgoOYRr3MC7kLNUMACTLw,i0cM1FAC3dy3t5tReCm98Q,NdpvGGF4cLrdnA6ydSZz3g,3,0,0,0,"good quality ingredients, horrible wait time.\...",2016-12-20 05:33:51


In [211]:
small_business_dataset_reviews = list(small_business_dataset['text'])

In [212]:
# convert the reviews into a concatenated string 
b1_review = ''.join(small_business_dataset_reviews)
clean_review = re.sub(r"[^A-Za-z0-9\s]+", "", b1_review)
b1_review = nlp(clean_review)

In [213]:
# removed punctuation and get the top 10 most common words (including stopwords)
b1_review_words = [token.text for token in b1_review if token.is_alpha == True] 
b1_word_freq = Counter(b1_review_words)
common_words = b1_word_freq.most_common(10)
print(common_words)

[('the', 619), ('and', 425), ('to', 404), ('I', 327), ('a', 311), ('was', 293), ('of', 272), ('for', 221), ('is', 205), ('we', 169)]


In [214]:
# removed punctuation and get the top 10 most common words (excluding stopwords)
b1_review_words = [token.text for token in b1_review if token.is_stop != True and token.is_alpha == True] 
b1_word_freq = Counter(b1_review_words)
common_words = b1_word_freq.most_common(10)
print(common_words)

[('nt', 129), ('food', 115), ('pot', 89), ('hot', 88), ('service', 87), ('soup', 87), ('place', 84), ('good', 74), ('table', 57), ('came', 56)]


In [215]:
#TODO: plot log graph


In [216]:
# now we do some stemming after removing the stopwords
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.stem import SnowballStemmer

porter_st = PorterStemmer()
lancaster_st = LancasterStemmer()
snow_st = SnowballStemmer("english")

In [217]:
# Using Porter Stemmer
porter_stemmed_words = [porter_st.stem(word) for word in b1_review_words]
porter_freq = Counter(porter_stemmed_words)
porter_common = porter_freq.most_common(10)
print(porter_common)

[('food', 132), ('nt', 129), ('place', 120), ('servic', 116), ('pot', 109), ('order', 102), ('soup', 97), ('hot', 93), ('good', 79), ('tabl', 69)]


In [218]:
# Using Lancaster Stemmer
lancaster_stemmed_words = [lancaster_st.stem(word) for word in b1_review_words]
lancaster_freq = Counter(lancaster_stemmed_words)
lancaster_common = lancaster_freq.most_common(10)
print(lancaster_common)

[('serv', 194), ('food', 132), ('nt', 129), ('plac', 120), ('pot', 112), ('ord', 102), ('soup', 97), ('hot', 93), ('wait', 80), ('good', 79)]


In [219]:
# Using Snowball Stemmer
snow_stemmed_words = [snow_st.stem(word) for word in b1_review_words]
snow_freq = Counter(snow_stemmed_words)
snow_common = snow_freq.most_common(10)
print(snow_common)

[('food', 132), ('nt', 129), ('place', 120), ('servic', 116), ('pot', 109), ('order', 102), ('soup', 97), ('hot', 93), ('good', 79), ('tabl', 69)]


### POS Tagging

In [220]:
random_sentences = reviews.sample(5, random_state=42)
random_sentences = list(random_sentences['text'])

In [221]:
random_sentences[0]

"Que ce soit pour leurs délicieux bubbles tea/smooties, leurs ''Bánh mì'' , leurs petits snacks (viennoiseries, tapioca, ...), on adore Vua et aussi leurs prix très abordables. On y retourne lorsqu'on est dans le Quartier Latin !"

In [222]:
nltk_tagged = []
for sentence in random_sentences:
    nltk_tagged.append((nltk.pos_tag(word_tokenize(sentence))))
nltk_tagged

[[('Que', 'NNP'),
  ('ce', 'NN'),
  ('soit', 'VBD'),
  ('pour', 'JJ'),
  ('leurs', 'NNS'),
  ('délicieux', 'VBP'),
  ('bubbles', 'NNS'),
  ('tea/smooties', 'NNS'),
  (',', ','),
  ('leurs', 'VBZ'),
  ('``', '``'),
  ('Bánh', 'NNP'),
  ('mì', 'NN'),
  ("''", "''"),
  (',', ','),
  ('leurs', 'VBZ'),
  ('petits', 'NNS'),
  ('snacks', 'NNS'),
  ('(', '('),
  ('viennoiseries', 'NNS'),
  (',', ','),
  ('tapioca', 'NN'),
  (',', ','),
  ('...', ':'),
  (')', ')'),
  (',', ','),
  ('on', 'IN'),
  ('adore', 'IN'),
  ('Vua', 'NNP'),
  ('et', 'CC'),
  ('aussi', 'JJ'),
  ('leurs', 'NNS'),
  ('prix', 'VBP'),
  ('très', 'JJ'),
  ('abordables', 'NNS'),
  ('.', '.'),
  ('On', 'IN'),
  ('y', 'JJ'),
  ('retourne', 'JJ'),
  ("lorsqu'on", 'NN'),
  ('est', 'JJS'),
  ('dans', 'NNS'),
  ('le', 'VBP'),
  ('Quartier', 'NNP'),
  ('Latin', 'NNP'),
  ('!', '.')],
 [('As', 'IN'),
  ('I', 'PRP'),
  ("'ve", 'VBP'),
  ('said', 'VBD'),
  ('previously', 'RB'),
  ('...', ':'),
  ('we', 'PRP'),
  ("'ve", 'VBP'),
  ('been

In [223]:
nlp = spacy.load("en_core_web_sm")
spacy_tagged = []
for sentence in random_sentences:
    spacy_tagged.append(nlp(sentence))
for tagged in spacy_tagged:
    for token in tagged:
        print(f'{token.text:{8}} {token.pos_:{6}}')

Que      PROPN 
ce       PROPN 
soit     ADJ   
pour     NOUN  
leurs    VERB  
délicieux NOUN  
bubbles  NOUN  
tea      NOUN  
/        SYM   
smooties NOUN  
,        PUNCT 
leurs    VERB  
''       PUNCT 
Bánh     PROPN 
mì       INTJ  
''       PUNCT 
,        PUNCT 
leurs    NOUN  
petits   VERB  
snacks   NOUN  
(        PUNCT 
viennoiseries NOUN  
,        PUNCT 
tapioca  INTJ  
,        PUNCT 
...      PUNCT 
)        PUNCT 
,        PUNCT 
on       ADP   
adore    PROPN 
Vua      PROPN 
et       PROPN 
aussi    PROPN 
leurs    VERB  
prix     NOUN  
très     ADJ   
abordables NOUN  
.        PUNCT 
On       ADP   
y        PROPN 
retourne VERB  
lorsqu'on PROPN 
est      PROPN 
dans     PROPN 
le       X     
Quartier PROPN 
Latin    PROPN 
!        PUNCT 
As       ADP   
I        PRON  
've      AUX   
said     VERB  
previously ADV   
...      PUNCT 
we've    PROPN 
been     AUX   
coming   VERB  
to       ADP   
LMAH     PROPN 
for      ADP   
over     ADP   
10       NUM 

# WORK COMPLETED UP TILL HERE.

### Writing Style

In [224]:
def getSOFurl():
    while(True):
        ip0 = 'stackoverflow'
        ip1 = 'com'
        ip2 = 'questions'
        ip3 = str(random.randint(0, 100000000))
        url = 'https://' + ip0 + '.' + ip1 + '/'+ ip2 + '/'+ ip3
        try:
            urlContent = urlopen(url).read()
            if urlContent:
                break
        except Exception as e: 
            print(e)
            pass

    print("Found URL: " + url)
    page1 = requests.get(url)
    soup1 = BeautifulSoup(page1.content, "html.parser")
    text = list(soup1.find_all("p"))
    sof_text = [txt.get_text() for txt in text]
   
    return sof_text


In [225]:
def getHWZurl():
    url = 'https://www.hardwarezone.com.sg/home'
    reqs = requests.get(url)
    soup = BeautifulSoup(reqs.text, 'html.parser')

    urls = []
    for link in soup.find_all('a'):
        urls.append(link.get('href'))
    random.shuffle(urls)
    while(True):
        for url in urls:
            print(url)
            try:
                url = 'https://www.hardwarezone.com.sg' + url
                urlContent = urlopen(url).read()
                page1 = requests.get(url)
                soup1 = BeautifulSoup(page1.content, "html.parser")
                text = list(soup1.find_all("p"))
                hwz_text = [txt.get_text() for txt in text]
                if urlContent and len(hwz_text)>10:
                    break
            except Exception as e: 
                print(e)
        break
    print("Found URL: " + url)
    return hwz_text

In [226]:
from urllib.parse import urlparse, urljoin
def getCNAurl():
    url = 'https://www.channelnewsasia.com/'
    reqs = requests.get(url)
    soup = BeautifulSoup(reqs.text, 'html.parser')

    linked_urls = set()

    def get_all_links(url):
        urls = set()
        domain_name = urlparse(url).netloc
        soup = BeautifulSoup(requests.get(url).content, "html.parser")
        for a_tag in soup.findAll("a"):
            href = a_tag.attrs.get("href")
            if href == "" or href is None:
                # href empty tag
                continue
            href = urljoin(url, href)
            parsed_href = urlparse(href)
            # remove URL GET parameters, URL fragments, etc.
            href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
            if href in urls:
                # already in the set
                continue
            if domain_name not in href:
                # external link
                continue
            if len(href) > 60:
                urls.add(href)
        return urls
    
    urls_cna = get_all_links('https://www.channelnewsasia.com/')
    url = random.sample(urls_cna, 1)
    print("Found URL: ", url[0])
    
    page1 = requests.get(cna_text1)
    soup1 = BeautifulSoup(page1.content, "html.parser")
    text = list(soup1.find_all("p"))
    cna_text = [txt.get_text() for txt in text]
   
    return cna_text


#### Cleaning text data

In [227]:
sof_text1 = getSOFurl()
sof_text2 = getSOFurl()

HTTP Error 404: Not Found
HTTP Error 404: Not Found
HTTP Error 404: Not Found
Found URL: https://stackoverflow.com/questions/68501807
Found URL: https://stackoverflow.com/questions/62830548


In [228]:
hwz_text1 = getHWZurl()
hwz_text2 = getHWZurl()

/features/list/30161-gallery
/features/list/37763-deals
https://forums.hardwarezone.com.sg
<urlopen error [Errno 11001] getaddrinfo failed>
/tech-news/list
/product-guide/23923-wearables/home
/priceLists
/tech-news-tpg-reveals-plans-trial-5g-nsa-year
Found URL: https://www.hardwarezone.com.sg/tech-news-tpg-reveals-plans-trial-5g-nsa-year
/product-guide/6580-projectors/home
/product-guide/261-casings/home
/priceLists
/product-guide/195-motherboard/home
/tech-news-thermaltake-pacific-pr32-d5-plus-reservoir-pump-combo-rgb-lighting
Found URL: https://www.hardwarezone.com.sg/tech-news-thermaltake-pacific-pr32-d5-plus-reservoir-pump-combo-rgb-lighting


In [229]:
cna_text1 = getCNAurl()
cna_text2 = getCNAurl()

Found URL:  https://www.channelnewsasia.com/commentary/commentary-singapores-response-against-potential-air-threats-well-oiled-machinery-task-remains-challenging-2176066


C:\Users\colot\AppData\Local\Temp/ipykernel_17396/3865290005.py:33: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  url = random.sample(urls_cna, 1)


InvalidSchema: No connection adapters were found for '[\'\\n\\n      World\\n  \\n\', \'\\n\\n      World\\n  \\n\', \'Seventh grade student Ryza Delos Santos, 10, works on her modules at home as her cousin observes, after a session at the makeshift rickshaw distance learning centre for the Aeta community in Porac, Pampanga, Philippines, Oct 12, 2020. (File photo: REUTERS/Eloisa Lopez)\', "MANILA: The United Nations children\'s agency UNICEF has urged education authorities to reopen schools as soon as possible in countries where millions of students are still not allowed to return to classrooms 18 months into the COVID-19 pandemic.", \'Schools in around 17 countries remain fully closed, while those in 39 countries remain partially closed, according to a report released by UNICEF on Thursday (Sep 16).\', \'Among those "almost completely closed" are schools usually attended by nearly 77 million students in the Philippines, Bangladesh, Venezuela, Saudi Arabia, Panama and Kuwait.\', "Nearl

##### Cleaning SOF text data

In [230]:
sof_text1 = sof_text1[4:]
sof_text1 = sof_text1[:-10]
temp_list = []
for line in sof_text1:
    temp_list += sent_tokenize(line)
sof_text1 = temp_list

In [231]:
sof_text1

['I have a table where records have a (begin, end) time window of existence (for things like employement duration, birth and death, rent duration, ...)',
 'begin IS NULL or end IS NULL if there is no bound.',
 'I want column value to be unique at any point in time.',
 'Is OK',
 'Whereas',
 'Is not OK, because both records have value=1 and overlapping time windows.',
 'Is there a way to enforce such a constraint in SQL ?',
 "You can't do this on the table, no, as there's nothing to make UNIQUE on.",
 'What you could do, however, is use a VIEW to enforce it.',
 "Firstly, let's create your table.",
 'I assume the columns datetime, should actually be begin and end; I recommend against these names as they are reserved keywords.',
 'As such I am calling them DateBegin and DateEnd.',
 'I am also assuming that they are date only (no time portion) values and so define them as a date not a datetime:',
 'And we\'ll INSERT your first 2 rows, as they are "ok":',
 'Now we need to make a VIEW, but we

In [232]:
sof_text2 = sof_text2[4:]
sof_text2 = sof_text2[:-10]
temp_list = []
for line in sof_text2:
    temp_list += sent_tokenize(line)
sof_text2 = temp_list

##### Cleaning CNA text data

In [233]:
cna_text1

['\n\n      World\n  \n',
 '\n\n      World\n  \n',
 'Seventh grade student Ryza Delos Santos, 10, works on her modules at home as her cousin observes, after a session at the makeshift rickshaw distance learning centre for the Aeta community in Porac, Pampanga, Philippines, Oct 12, 2020. (File photo: REUTERS/Eloisa Lopez)',
 "MANILA: The United Nations children's agency UNICEF has urged education authorities to reopen schools as soon as possible in countries where millions of students are still not allowed to return to classrooms 18 months into the COVID-19 pandemic.",
 'Schools in around 17 countries remain fully closed, while those in 39 countries remain partially closed, according to a report released by UNICEF on Thursday (Sep 16).',
 'Among those "almost completely closed" are schools usually attended by nearly 77 million students in the Philippines, Bangladesh, Venezuela, Saudi Arabia, Panama and Kuwait.',
 "Nearly a third of this figure is accounted for by the Philippines, which

##### Cleaning HWZ text data

In [234]:
hwz_text1 = hwz_text1[:-5]
temp_list = []
for line in hwz_text1:
    temp_list += sent_tokenize(line)
hwz_text1 = temp_list

In [235]:
hwz_text2 = hwz_text2[:-5]
temp_list = []
for line in hwz_text2:
    temp_list += sent_tokenize(line)
hwz_text2 = temp_list

##### First word in sentence capitalized?

In [236]:
count=0
uppercount=0
for sent in sof_text1:
    if sent[0].isupper():
        uppercount+=1
    count+=1
print("Fraction of first letter being capitalised for sof_text1: ", uppercount/count)

Fraction of first letter being capitalised for sof_text1:  0.9259259259259259


In [237]:
count=0
uppercount=0
for sent in sof_text2:
    if sent[0].isupper():
        uppercount+=1
    count+=1
print("Fraction of first letter being capitalised for sof_text2: ", uppercount/count)

Fraction of first letter being capitalised for sof_text2:  0.9333333333333333


In [238]:
count=0
uppercount=0
for sent in hwz_text1:
    if sent[0].isupper():
        uppercount+=1
    count+=1
print("Fraction of first letter being capitalised for hwz_text1: ", uppercount/count)

Fraction of first letter being capitalised for hwz_text1:  1.0


In [239]:
count=0
uppercount=0
for sent in hwz_text2:
    if sent[0].isupper():
        uppercount+=1
    count+=1
print("Fraction of first letter being capitalised for hwz_text2: ", uppercount/count)

Fraction of first letter being capitalised for hwz_text2:  0.9523809523809523


In [241]:
count=0
uppercount=0
for sent in cna_text1:
    if sent[0].isupper():
        uppercount+=1
    count+=1
print("Fraction of first letter being capitalised for cna_text1: ", uppercount/count)

Fraction of first letter being capitalised for cna_text1:  0.6818181818181818


##### Length of articles?

In [242]:
print("No of sentences in sof_text1: ", len(sof_text1))
print("No of sentences in sof_text2: ", len(sof_text2))
print("No of sentences in hwz_text1: ", len(hwz_text1))
print("No of sentences in hwz_text2: ", len(hwz_text2))

No of sentences in sof_text1:  27
No of sentences in sof_text2:  15
No of sentences in hwz_text1:  15
No of sentences in hwz_text2:  21


##### Proper nouns capitalised?

In [ ]:
tagged = []
uppercount = 0
count = 0
for sentence in sof_text1:
    tagged.append(nlp(sentence))
for tag in tagged:
    for token in tag:
        if token.pos_ == 'PROPN':
            if token.text[0].isupper():
                uppercount += 1
            count += 1
print('Fraction of proper nouns capitalised in sof_text1: ', uppercount/count)

In [ ]:
tagged = []
uppercount = 0
count = 0
for sentence in sof_text2:
    tagged.append(nlp(sentence))
for tag in tagged:
    for token in tag:
        if token.pos_ == 'PROPN':
            if token.text[0].isupper():
                uppercount += 1
            count += 1
print('Fraction of proper nouns capitalised in sof_text2: ', uppercount/count)

In [ ]:
tagged = []
uppercount = 0
count = 0
for sentence in hwz_text1:
    tagged.append(nlp(sentence))
for tag in tagged:
    for token in tag:
        if token.pos_ == 'PROPN':
            if token.text[0].isupper():
                uppercount += 1
            count += 1
print('Fraction of proper nouns capitalised in hwz_text1: ', uppercount/count)

In [ ]:
tagged = []
uppercount = 0
count = 0
for sentence in hwz_text2:
    tagged.append(nlp(sentence))
for tag in tagged:
    for token in tag:
        if token.pos_ == 'PROPN':
            if token.text[0].isupper():
                uppercount += 1
            count += 1
print('Fraction of proper nouns capitalised in hwz_text2: ', uppercount/count)

###### What kind of proper nouns used?

1. Stack Overflow

In [ ]:
sof_tagged = []
sof_noun_dict = {}
for sentence in sof_text1:
    sof_tagged.append(nlp(sentence))
for sentence in sof_text2:
    sof_tagged.append(nlp(sentence))
for tagged in sof_tagged:
    for token in tagged:
        if token.pos_ in ("PROPN", "NOUN"):
            if token.text in sof_noun_dict.keys():
                sof_noun_dict[token.text] += 1
            else:
                sof_noun_dict[token.text] = 1

sof_noun_dict_sorted = sorted(sof_noun_dict.items(), key=lambda x: x[1], reverse=True)

for i in sof_noun_dict_sorted:
    print(i[0], i[1])

In [ ]:
hwz_tagged = []
hwz_noun_dict = {}
for sentence in hwz_text1:
    hwz_tagged.append(nlp(sentence))
for sentence in hwz_text2:
    hwz_tagged.append(nlp(sentence))
for tagged in hwz_tagged:
    for token in tagged:
        if token.pos_ in ("PROPN", "NOUN"):
            if token.text in hwz_noun_dict.keys():
                hwz_noun_dict[token.text] += 1
            else:
                hwz_noun_dict[token.text] = 1

hwz_noun_dict_sorted = sorted(hwz_noun_dict.items(), key=lambda x: x[1], reverse=True)

for i in hwz_noun_dict_sorted:
    print(i[0], i[1])

##### Good grammar?
1. Tense matching
2. Subject-verb agreement

#### Subject-verb agreement

In [ ]:
# from https://github.com/pes10k/cs412-scorer/blob/master/agreement_utils.py
singular_noun_tags = ('NN', 'NNP')
plural_noun_tags = ('NNS', 'NNPS')

noun_tags = singular_noun_tags + plural_noun_tags + ('PRP',)

plural_verb_tags = ('VBZ',)
singular_verb_tags = ('VBP',)
general_verb_tags = ('VBD', 'VB')

verb_tags = singular_verb_tags + plural_verb_tags + general_verb_tags

plural_prop_nouns = ('they', 'we', 'them', 'themselves', 'us', 'those')
singular_prop_nouns = ('he', 'she', 'i', 'him', 'me', 'myself', 'it')

singulars = singular_noun_tags + singular_verb_tags + singular_prop_nouns
plurals = plural_noun_tags + plural_verb_tags + plural_prop_nouns

# HELP!

In [ ]:
agreed = 0
disagreed = 0
agree_pairs = []
disagree_pairs = []
for sentence in sof_text1:
    sentence = sentence.lower().split()
    tagged = nltk.pos_tag(sentence)
    for index, word_tag_pair in enumerate(tagged):
        if word_tag_pair[1] in singulars:
            for j in range(index, len(tagged)):
                if tagged[j][1] in singulars:
                    agree_pairs.append((word_tag_pair[0], tagged[j][0]))
                    agreed += 1
                elif tagged[j][1] in plurals:
                    disagree_pairs.append((word_tag_pair[0], tagged[j][0]))
                    disagreed += 1
        elif word_tag_pair[1] in plurals:
            for j in range(index, len(tagged)):
                if tagged[j][1] in singulars:
                    disagree_pairs.append((word_tag_pair[0], tagged[j][0]))
                    disagreed += 1
                elif tagged[j][1] in plurals:
                    agree_pairs.append((word_tag_pair[0], tagged[j][0]))
                    agreed += 1
print(agree_pairs)
print(disagree_pairs)
print("SVA ratio in sof_text1: ", agreed/(agreed+disagreed))
                    
            

# So i learnt that this way is not good. i will attempt to try with spacy dependency tracker and then combine with nltk pos tagger to check if there is SVA. thx

# this is what i will do
# 1. choose 6 good urls (2 from each source) so that there can be comparisons
# 2. do the CNA preprocessing part
# 3. continue with the two parts for checking for good grammar (any more ideas?)
# 4. think n implement more things to analyse for writing style
